# Beam search

In [1]:
import torch
import torch.nn.functional as F

In [ ]:
def beam_search(decoder, hidden, context, beam_width=3, max_length=10):
    
    sequences = [[[], 1.0, hidden]] # 초기화
    # [["나는", 0.6, hidden1],
    #   ["저는", 0.4, hidden2]]  # 문장, 확률, hidden state를 담고 있는 배열을 만든다.

    for _ in range(max_length):
        all_candidates = [] # 후보군 유지를 해준다는 특징 떄문에 문장을 저장해 줄 배열을 만든다.
        for seq, score, hidden in sequences:
            decoder_input = torch.tensor([seq[-1] if seq else 0])
            output, hidden = decoder(decoder_input, hidden, context)
            top_probs, top_indices = torch.topk(F.softmax(output, dim=1), beam_width)   # 상위 beam_width 개수의 확률과 인덱스를 가져온다 (torch.topk)

            for i in range(beam_width):
                candidate = (seq + [top_indices[0][i].item()], score * top_probs[0][i].item(), hidden)
                all_candidates.append(candidate)

        sequences = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width] # 후보군을 확률값이 높은 순으로 정렬하여 상위 beam_width 개수만 남긴다.
        
    return sequences[0][0] # 가장 높은 확률을 가진 문장을 반환한다.

